In [ ]:
from IPython.display import HTML

HTML('''<script>

code_show=true; 

function code_toggle() {
if (code_show){
$('div.input').hide();
    if (document.getElementById('input') !== null)
        {document.getElementById('input').value = 'Display code in notebook';}
} else {
$('div.input').not(':first').show();
document.getElementById('input').value = 'Hide code in notebook';
}
code_show = !code_show
}

$( document ).ready(code_toggle);
$('div.cell.code_cell.rendered.selected').find('div.input').hide();



</script>

<form id="form" action="javascript:code_toggle()">
    <input style="position: fixed;top: 200;right: 0;" id="input" type="submit" value="Display code in notebook">
</form>


    


''')

In [ ]:
import dask.dataframe as dd
import os
import pandas as pd
import numpy as np
from dask.distributed import Client
import geopandas as gpd
import matplotlib.pyplot as plt
import hvplot.pandas
import sys
import xarray as xr
import warnings
from IPython.display import Markdown as md
import cartopy.crs as ccrs
import hvplot
import holoviews as hv
from geoviews import opts
import geoviews as gv
import geoviews.feature as gf
import s3fs
import fsspec
from bokeh.layouts import widgetbox
from bokeh.layouts import column as bokehCol
from bokeh.models.layouts import Column
from bokeh.models.widgets import DatePicker
from datetime import date
import panel as pn
import hvplot.xarray

pd.set_option('display.max_rows', 2000)
warnings.filterwarnings("ignore")

# Local functions
import sefm.backends.climate.storm_typing as st
import sefm.backends.climate.zones as zn

import sefm.utils.hydrotools as ht

In [ ]:
# Dask client to implement distributed calculations (multicore or multinode if a cluster is available)
client = Client(silence_logs='error')

# Introduction

## Usage

Storm typing classifies storms events into classes that are nearly homogeneous with respects to their generating phenomenon. This allows to evaluate more homogeunous distributions for each storm type when computing precipitation-frequency analysis rather than a traditionel mixed distribution with all the storms. Homogeneity is an assumption when computing frequency analysis which makes storm typing all the more important.

## Storm types

There are three main storm types that can arise over the Outaouais watershed: 
- Mid-latitude cyclones (MLC),
- Mesoscale storms (MEC)
- Local storms (LS). 

These storm types have unique spatiotemporal characteristics which makes them nearly humogeneous for frequency analysis purposes. Specifically for the Outaouais watershed, the MLC and MLC/MEC hydrid storm types have the most impact as their duration is ofter longer than 2 days and because they usually cover a large area size which can lead to important flood volume in the watershed.

| Storm Type and Sub-Type                       | Acronym                 | Numerical Code |
|-----------------------------------------------|-------------------------|----------------|
| Mid-Latitude Cyclone                          | MLC                     | 10             |
| Mid-Latitude Cyclone with Embedded Convection | MLC/EC                  | 13             |
| Mesoscale Storm                               | MLC/EC                  | 30             |
| Mesoscale Storm with Embedded Convection      | MLC/EC                  | 33             |
| Local Storm                                   | MLC/EC                  | 40             |
| Local Storm with Enhanced Convection          | LS/LEC                  | 43             |
| MLC/MEC Hybrid                                |                         | 60             |
| MLC/MEC Hybrid with Embedded Convection       |                         | 63             |
| Dry Day                                       |                         | 99             |
|                                               |                         |                |

## Database of Daily Storm Types (DDST)

Description ...

# 1.1 Storm typing

## 1.1.1 Domain

Two (2) domains are proposed to study precipitation in the Outaouais region and both of their pixels are aligned with the [20th century reanalysis V3's](https://psl.noaa.gov/data/gridded/data.20thC_ReanV3.html) pixels resolution:
- Domain 1: A corser grid based on TVA's report. (4 zones)
- Domain 2: A finer grid based on NB Power' report. (40 zones)

In [ ]:
zones_definition = {'zone_1' : {'filenames':['grid-large.geojson', # polygon for coarse grids
                                             'lines-large.geojson']}, # lines for coarse grids
                    'zone_2' : {'filenames':['grid-small.geojson']}, # polygon for fine grids
                    'background' : {'filenames': ['Bassins Outaouais HSAMI.geojson']}} # watershed in background

zones = zn.Zones(zones=zones_definition,
                 bucket='s3://sefm/gis',
                 client_kwargs={"endpoint_url":"https://s3.us-west-1.wasabisys.com"})

zones.interactive_map()

In [ ]:
region = 'zone_2_id'

domain_dict = {'zone_1_id': 'Domain 1',
               'zone_2_id': 'Domain 2'}

md("""Let's select %s to begin with..."""%(str(domain_dict[region])))

## 1.1.2 Manual storm typing

In [ ]:
md("""For each zone inside the %s, the following variables can be calculated. 
Calculation for each data point is demonstrated in the following sub-sections :"""%(str(domain_dict[region])))

**Global Historical Climatology Network (GHCN)** :
- cumulative precipitation (72h)

**20th Century Reanalysis V3** :
- precipitation rate;
- cloud cover;
- convective available potential energy;
- precipitable water;
- gradient in the 500-mb height;
- gradient in the 850-mb height.

Let's also use the **1979-09-14 - 1979-09-16** historical storm to vizualise all the variables.


### 1.1.2.1 Cumulative Precipitation (72h)

The Global Historical Climatology Network (GHCN) is an integrated database of climate summaries from land surface stations across the globe that have been subjected to a common suite of quality assurance reviews. We use this database to extract cumulated precipitation over a specified duration for each stations available over a specified region.

In [ ]:
# Information on ghcnd's bucket
bucket = 's3://ghcnd-can-us-ne'
storage_options={'anon': True,
                 "client_kwargs": {'endpoint_url': 'https://s3.us-east-2.wasabisys.com'}}

In [ ]:
text = """
The data can be acquired from the following sources :
- bucket : %s
- endpoint_url : %s
"""%(bucket, storage_options['client_kwargs']['endpoint_url'])

md(text)

We can also define a bounding box to limit the metadata extraction to our study zone :

In [ ]:
latlngbox = [-82, -74, 44.5, 49]

In [ ]:
md("""- Bounding box = %s  - *(lon_min, lon_max, lat_min, lat_max)*"""%(str(latlngbox)))

All stations available within our bounding box can be viewed in the following table : 

In [ ]:
stations = st.Stations(metadata_bucket=os.path.join(bucket,'ghcdn_stations.csv'),
                       storage_options=storage_options)

In [ ]:
stations.read_metadata(latlngbox=latlngbox);

In [ ]:
df = stations.read_parquet(os.path.join(bucket,'data/parquet/data.parquet'),
                           element='PRCP',
                           storage_options=storage_options)
df['value'] = df['value']/10.0 # To convert into mm

In [ ]:
df_pivot = df.pivot_table(index='date', columns='id', values='value')
df_pivot.index = pd.to_datetime(df_pivot.index)

df_pivot = df_pivot[(df_pivot.index.year>=1900) & (df_pivot.index.year<2020)]
df_pivot = df_pivot.rolling('3D').sum().round(3)

In [ ]:
# Correction for missing data
stations.metadata = stations.metadata[~stations.metadata.id.isin(df_pivot.columns[df_pivot.count()==0].values)]

stations.metadata = stations.metadata[~stations.metadata.id.isin(list(set(stations.metadata.id.values) - 
                                                                      set(df_pivot.columns)))]

In [ ]:
gdf_stations = gpd.GeoDataFrame(stations.metadata, 
                                geometry=gpd.points_from_xy(stations.metadata.longitude,
                                                            stations.metadata.latitude))

Spatiotemporal precipitation data availability can also be vizualised :

In [ ]:
stations.plot_stations() + \
df_pivot.count(axis=1).resample('1Y').mean().hvplot(kind='scatter', grid=True,
                                                   ylabel='Number of operating weather stations (rain)',
                                                    width=600, height=400,
                                                   title = 'Operating weather stations in the Outaouais region')

In [ ]:
ddst = pd.DataFrame(columns = ['date','zone_id','variable','value'])

In [ ]:
threshold = 1.58

zone_1_gdf = zones.data['zone_1']['gdf'][zones.data['zone_1']['gdf'].geometry.type.isin(['MultiPolygon'])].set_index('id')
zone_2_gdf = zones.data['zone_2']['gdf'][zones.data['zone_2']['gdf'].geometry.type.isin(['Polygon'])].set_index('id')

for index, zone_id in zone_1_gdf.iterrows():
    stations.metadata.loc[stations.metadata.index.isin(gdf_stations[gdf_stations.within(zone_id.geometry)].index),
                      'zone_1_id'] = index
for index, zone_id in zone_2_gdf.iterrows():
    stations.metadata.loc[stations.metadata.index.isin(gdf_stations[gdf_stations.within(zone_id.geometry)].index),
                      'zone_2_id'] = index

data = []

for zone_id in sorted(stations.metadata[region].dropna().unique()):
    df_pivot_zone = df_pivot.loc[:,stations.metadata[stations.metadata[region].isin([zone_id])].id]
    
    # Maximum precipitation
    series = df_pivot_zone.max(axis=1)
    dict_max_precip = {'date': series.index, 
                       'zone_id': int(zone_id),
                       'variable': 'max_precip',
                       'value': series.values}

    data.extend([dict_max_precip])


In [ ]:
max_precip = zone_2_gdf.join(pd.concat([pd.DataFrame.from_dict(values) for values in data]).set_index('zone_id'))
max_precip['longitude'] = max_precip.centroid.x
max_precip['latitude'] = max_precip.centroid.y
max_precip = max_precip.rename(columns={'date': 'time'})
max_precip = max_precip.set_index(['time','longitude', 'latitude'])
da_max_precip = xr.DataArray.from_series(max_precip['value'])

Extremes storms events can therefore be displayed for the zones of Domain 2. For instance, the 1979-09-13 - 1979-09-15 72h total precipitation is shown below :

In [ ]:
da_values = da_max_precip.sel(time='1979-09-15')
df_da = da_values.to_dataframe().reset_index()

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da['value']]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 :72h total cumulative precipitation')*labels

### 1.1.2.2 Precipitation rate

In [ ]:
bucket = 's3://cires-20-century-reanalysis-v3/zarr/single-levels'

storage_options = {'endpoint_url': 'https://s3.us-east-1.wasabisys.com'}

ds = xr.open_zarr(fsspec.get_mapper(bucket,
                                    client_kwargs=storage_options,
                                    anon=True),
                  consolidated=True)

ds_large_scale = ds

ds = ds.sel(time=slice('1900-01-01', '2015-12-31'),
            longitude=slice(latlngbox[0], latlngbox[1]),
            latitude=slice(latlngbox[2], latlngbox[3]))


In [ ]:
ds['apcp72'] = da_max_precip

In [ ]:
variable='prate'

da_values = ds[variable].sel(time=slice('1979-09-13','1979-09-15')).max('time')

da_values = da_values*3600

df_da = da_values.to_dataframe().reset_index()

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da[variable]]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 : maximum precipitation rate (72h window)')*labels

### 1.1.2.3 Cloud cover

In [ ]:
variable='tcdc'

da_values = ds[variable].sel(time=slice('1979-09-13','1979-09-15')).max('time')

df_da = da_values.to_dataframe().reset_index()

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da[variable]]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 : maximum cloud cover (72h window)')*labels

### 1.1.2.4 Convective potential available energy

In [ ]:
variable='cape'
import geoviews.feature as gf
# da_values = ds[variable].sel(time=slice('1979-09-14','1979-09-16')).max('time')
da_values = ds[variable].sel(time=slice('1979-09-13','1979-09-15')).max('time')

df_da = da_values.to_dataframe().reset_index()

# data = list(zip(zone_plot.Polygons.values()[0].data.geometry.centroid.x,
#                                         zone_plot.Polygons.values()[0].data.geometry.centroid.y))

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da[variable]]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 : maximum CAPE (72h window)')*labels
# zones.data['background']['gdf'].hvplot(alpha=0.1)

In [ ]:
ds_hvplot = ds_large_scale[variable].sel(time=slice('1979-09-13','1979-09-15'))\
                        .max('time')\
# enlever le tooltip
ds_hvplot.hvplot.contourf(levels=30,
                          project=True,
                          grid=True,
                          hover=False,
                          tiles='EsriUSATopo',
                          alpha=0.2,
                          cmap='rainbow')*\
ds_hvplot.hvplot.contour(levels=30,
                          project=True,
                          grid=True,
                          alpha=1,
                          cmap='rainbow')

### 1.1.2.5 Precipitable water

In [ ]:
variable='pr_wtr'

da_values = ds[variable].sel(time=slice('1979-09-13','1979-09-15')).max('time')

df_da = da_values.to_dataframe().reset_index()

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da[variable]]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 : maximum precipitable water (72h window)')*labels

In [ ]:
ds_hvplot = ds_large_scale[variable].sel(time=slice('1979-09-13','1979-09-15'))\
                        .max('time')\
# enlever le tooltip
ds_hvplot.hvplot.contourf(levels=30,
                          project=True,
                          grid=True,
                          hover=False,
                          tiles='EsriUSATopo',
                          alpha=0.2,
                          cmap='rainbow')*\
ds_hvplot.hvplot.contour(levels=30,
                          project=True,
                          grid=True,
                          alpha=1,
                          cmap='rainbow')

### 1.1.2.6 500 mb Gradients

In [ ]:
bucket = 's3://cires-20-century-reanalysis-v3/zarr/pressure-levels'

storage_options = {'endpoint_url': 'https://s3.us-east-1.wasabisys.com'}

ds_pl = xr.open_zarr(fsspec.get_mapper(bucket,
                                    client_kwargs=storage_options,
                                    anon=True),
                     consolidated=True)

ds__pl_large_scale = ds_pl

ds_pl = ds_pl.sel(time=slice('1900-01-01', '2015-12-31'),
                  longitude=slice(latlngbox[0], latlngbox[1]),
                  latitude=slice(latlngbox[2], latlngbox[3]))


In [ ]:
da = np.sqrt(np.square(ds_pl.sel(level=500).hgt.differentiate('longitude')) + 
             np.square(ds_pl.sel(level=500).hgt.differentiate('latitude')))
ds_pl['gradh500'] = da

In [ ]:
variable = 'gradh500'

da_values = ds_pl[variable].sel(time=slice('1979-09-13','1979-09-15')).max('time')

df_da = da_values.to_dataframe().reset_index()

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da[variable]]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 : 500 mb maximum gradient (72h window)')*labels

In [ ]:
ds_hvplot = ds__pl_large_scale['hgt'].sel(level=500, time=slice('1979-09-13','1979-09-15'))\
                                        .max('time')\
# enlever le tooltip
ds_hvplot.hvplot.contourf(levels=30,
                          project=True,
                          grid=True,
                          hover=False,
                          tiles='EsriUSATopo',
                          title='1979-09-15 : 500 mb maximum (72h window)',
                          alpha=0.2,
                          cmap='rainbow')*\
ds_hvplot.hvplot.contour(levels=30,
                          project=True,
                          grid=True,
                          alpha=1,
                          cmap='rainbow')

### 1.1.2.6 900 mb Gradients

In [ ]:
da = np.sqrt(np.square(ds_pl.sel(level=900).hgt.differentiate('longitude')) + 
             np.square(ds_pl.sel(level=900).hgt.differentiate('latitude')))
ds_pl['gradh900'] = da

In [ ]:
variable = 'gradh900'

da_values = ds_pl[variable].sel(time=slice('1979-09-13','1979-09-15')).max('time')

df_da = da_values.to_dataframe().reset_index()

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da[variable]]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 : 900 mb maximum gradient (72h window)')*labels

In [ ]:
ds_hvplot = ds__pl_large_scale['hgt'].sel(level=900, time=slice('1979-09-13','1979-09-15'))\
                                        .max('time')\
# enlever le tooltip
ds_hvplot.hvplot.contourf(levels=30,
                          project=True,
                          grid=True,
                          hover=False,
                          tiles='EsriUSATopo',
                          title='1979-09-15 : 900 mb maximum (72h window)',
                          alpha=0.2,
                          cmap='rainbow')*\
ds_hvplot.hvplot.contour(levels=30,
                          project=True,
                          grid=True,
                          alpha=1,
                          cmap='rainbow')

### 1.1.3.8 Preliminary type

In [ ]:
# Verify the rolling period
ds_rolling_sl = ds[['cape','prate']].rolling(time=24).max().resample(time='1D').max('time')

In [ ]:
ds_rolling_pl = ds_pl[['gradh500','gradh900']].rolling(time=24).max().resample(time='1D').max('time')

In [ ]:
# ruled-based algorithm

# da = xr.where((ds_roll.prate<0.12) , 99, -1)

# 1. 0.12 < Precipitation rate <= 0.5 and CAPE < 500
a = xr.ufuncs.logical_and(ds_rolling_sl.prate <= 0.5/3600, ds_rolling_sl.cape<500)
b = xr.ufuncs.logical_and(ds_rolling_sl.prate >0.12/3600, a)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 40, -1)

# 2. 0.12 < Precipitation rate <= 0.5 and CAPE >= 500
a = xr.ufuncs.logical_and(ds_rolling_sl.prate <= 0.5/3600, ds_rolling_sl.cape>=500)
b = xr.ufuncs.logical_and(ds_rolling_sl.prate >0.12/3600, a)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 40, da)

# 3. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient < 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900<8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 30, da)

# 4. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient < 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900<8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 33, da)

# 5. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient >= 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900>=8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 10, da)

# 6. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient >= 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900>=8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 13, da)

# 7. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient >= 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900>=8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 60, da)

# 8. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient >= 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900>=8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 63, da)

# 9. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient < 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900<8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 60, da)

# 10. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient < 8
a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900<8)
c = xr.ufuncs.logical_and(a,b)
da = xr.where(c, 63, da)

# 10. Precipitation rate <0.12
da = xr.where(ds_rolling_sl.prate<=0.12/3600, 99, da)

In [ ]:
variable = 'prelim_type'

da_values = da.sel(time='1979-09-15')

df_da = da_values.to_dataframe(name=variable).reset_index()

labels = hv.Labels({('x', 'y'): df_da[['longitude','latitude']],
                    'text': [i for i in df_da[variable]]},
                                           ['x', 'y'], 'text') 

da_values.hvplot(x='longitude', y='latitude', cmap='isolum',
                title='1979-09-15 : Preliminary storm type')*labels

### 1.1.3 Automated storm typing algorithm

FROM THIS POINT ON, THIS IS A WORK IN PROGRESS